In [1]:
import numpy as np
import pandas as pd
from sklearn.svm import OneClassSVM
from utils import AudioSVMClassifier
import os

In [7]:
base_directory = "/data/Deep_Fake_Data/"
feat_directory = "Features_no_padding"
data_names = 'ASV19'
data_types = "train"
# features_ls  = ['ssast_patch_base', 'wavlm_large', 'data2vec_large_ll60k', 'wavlablm_mk_40k', 'unispeech_sat_large', 'hubert_large_ll60k', 'xlsr_53', 'passt_base', 'espnet_hubert_base_iter0', 'mae_ast_patch', 'ast', 'byol_a_2048', 'modified_cpc', 
#                  'byol_s_cvt', 'wav2vec2_base_s2st_es_voxpopuli']
features_ls  = ['ssast_patch_base']
speakers_ls = ['p229']
deepfake_types  = ['A01','A02']
output_dir = 'one_class_svm'

In [8]:
nu=0.05
gamma='auto'

In [9]:
for speaker in speakers_ls:
    for deepfake in deepfake_types:
        for feature in features_ls:
            path_to_original_data = os.path.join(base_directory, feat_directory, data_names, speaker, data_types, 'Original', feature)
            classifier = AudioSVMClassifier(speaker, base_directory, feature, output_dir)
            classifier.train(path_to_original_data, nu, gamma)
            path_to_deepfake_data = os.path.join(base_directory, feat_directory, data_names, speaker, data_types, deepfake, feature)
            y_orig, y_pred = classifier.test(path_to_deepfake_data, )
            results = classifier.evaluate_model(compute_accuracy=True, compute_auc=True, compute_conf_matrix=False)


one_class_svm/p229


/home/suryasss/.conda/envs/test_gpu/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1183: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(


one_class_svm/p229


/home/suryasss/.conda/envs/test_gpu/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1183: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(


In [10]:
results

{'accuracy': 100.0, 'auc': np.float64(nan)}

In [2]:
from e2e_utils import AudioSVMClassifier

/home/suryasss/.conda/envs/test_gpu/lib/python3.10/site-packages/s3prl/upstream/byol_s/byol_a/common.py:20: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("sox_io")
/home/suryasss/.conda/envs/test_gpu/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import os
import pickle
import numpy as np
import torch
import torchaudio
from s3prl.nn import S3PRLUpstream, Featurizer
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.metrics import confusion_matrix, accuracy_score, roc_curve, auc
from sklearn.model_selection import train_test_split

In [5]:
if __name__ == "__main__":
    # Define device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Initialize the detector with a chosen S3PRL model
    model_name = "wavlm_large"
    detector = AudioSVMClassifier(model_name, device, speaker="Donald_Trump")
    
    # Suppose we have a folder of audio files for training.
    training_folder = "/data/FF_V2/Famous_Figures/Donald_Trump/Original"  # Update this path to your folder
    detector.train_svm_from_folder(training_folder, nu=0.1, gamma=0.1, aggregate_emb=False,layer_number=0)
    
    # Optionally, save the trained scaler and SVM model for later use.
    saved_model_dir = "saved_model"
    os.makedirs(saved_model_dir, exist_ok=True)
    with open(os.path.join(saved_model_dir, "scaling_model.pkl"), "wb") as f:
        pickle.dump(detector.scaler, f)
    with open(os.path.join(saved_model_dir, "svm_model.pkl"), "wb") as f:
        pickle.dump(detector.svm_model, f)
    
    # Example: Use the predict_from_audio method to predict on a new audio file.
    test_audio_file = "/data/FF_V2/Famous_Figures/Donald_Trump/spoof/Donald_Trump_00001_FISHSPEECH_616_0.5.wav"  # Update this path to your test file
    prediction = detector.predict_from_audio(test_audio_file, aggregate_emb=True, load_model_dir=saved_model_dir)
    print("Prediction for the test audio file:", prediction)


Using device: cuda
Loading model: wavlm_large


/home/suryasss/.conda/envs/test_gpu/lib/python3.10/site-packages/s3prl/upstream/wavlm/expert.py:37: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckp

Model wavlm_large loaded on cuda
Extracted features from Donald_Trump_00001.wav
Extracted features from Donald_Trump_00002.wav
Extracted features from Donald_Trump_00003.wav
Extracted features from Donald_Trump_00005.wav
Extracted features from Donald_Trump_00007.wav
Extracted features from Donald_Trump_00008.wav
Extracted features from Donald_Trump_00009.wav
Extracted features from Donald_Trump_00011.wav
Extracted features from Donald_Trump_00012.wav
Extracted features from Donald_Trump_00013.wav
Extracted features from Donald_Trump_00014.wav
Extracted features from Donald_Trump_00015.wav
Extracted features from Donald_Trump_00017.wav
Extracted features from Donald_Trump_00018.wav
Extracted features from Donald_Trump_00019.wav
Extracted features from Donald_Trump_00021.wav
Extracted features from Donald_Trump_00022.wav
Extracted features from Donald_Trump_00024.wav
Extracted features from Donald_Trump_00026.wav
Extracted features from Donald_Trump_00028.wav
Extracted features from Don

KeyboardInterrupt: 